In [227]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelBinarizer
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.linear_model import SGDOneClassSVM
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.decomposition import PCA
from sklearn.kernel_approximation import Nystroem
from sklearn import metrics
from sklearn.svm import OneClassSVM
import time
import copy
import re
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
import os,sys
import tqdm
import pickle
import time
from sklearn.preprocessing import StandardScaler
import ast
from sklearn.pipeline import make_pipeline
import torch as T
# import torch.utils.data as data

In [228]:
dataPath = 'D:/mt_data/data/'
feature ='system calls tfidf_1gram'
filename = 'encoded_bow_{}.csv'.format(feature)
t1 = dataPath+filename
t1df = pd.read_csv(t1, sep='\t')
t1data = [ast.literal_eval(i) for i in t1df['data']]
data = pd.DataFrame([t1df['device'].tolist(), t1df['label'].tolist(), t1df['malware'].tolist(), t1data]).transpose()
data.columns = ['device', 'label','malware', 'data']

In [229]:
traindevice = 'pi4_4G_t1'
test1device = 'pi3_t1'
test2device = 'pi4_2G_t1'
td = 'pi4_4G'
test3device = 'pi3_t2'
test4device = 'pi4_2G_t2'
test5device = 'pi4_4G_t2'

In [230]:
scaledPath =  'd:/mt_data/1126_withrw/encoded/t1/' + '{}_{}_{}-scaled.pk'.format(td, 60, feature)
loc = open(scaledPath, 'rb')
scaled = pickle.load(loc)

In [231]:
X = data[(data['device']==traindevice)&(data.malware=='normal')]['data'].tolist()

In [232]:
X = scaled.transform(X).tolist()
y = [1 for i in range(0, len(X))]
X_train, X_val, y_train, y_val = train_test_split(X, y, shuffle=False, test_size=.3)


In [233]:
X_test = data[(data['device']==test1device)&(data.label==-1)]['data'].tolist()
X_test = scaled.transform(X_test).tolist()
y_test =  [-1 for i in range(0, len(X_test))]

In [234]:
test1_normal = data[(data['device']==test1device)&(data.malware=='normal')]['data'].tolist()
test1_normal = scaled.transform(test1_normal).tolist()

test1_attack = data[(data['device']==test1device)&(data.label==-1)]['data'].tolist()
test1_attack = scaled.transform(test1_attack).tolist()
y1_normal =  [1 for i in range(0, len(test1_normal))]
y1_attack =  [-1 for i in range(0, len(test1_attack))]

In [235]:
test2_normal = data[(data['device']==test2device)&(data.malware=='normal')]['data'].tolist()
test2_normal = scaled.transform(test2_normal).tolist()
test2_attack = data[(data['device']==test2device)&(data.label==-1)]['data'].tolist()
test2_attack = scaled.transform(test2_attack).tolist()
y2_normal =  [1 for i in range(0, len(test2_normal))]
y2_attack =  [-1 for i in range(0, len(test2_attack))]

In [236]:
test3_normal = data[(data['device']==test3device)&(data.malware=='normal')]['data'].tolist()
test3_normal = scaled.transform(test3_normal).tolist()
y3_normal =  [1 for i in range(0, len(test3_normal))]

In [237]:
test4_normal = data[(data['device']==test4device)&(data.malware=='normal')]['data'].tolist()
test4_normal = scaled.transform(test4_normal).tolist()
y4_normal =  [1 for i in range(0, len(test4_normal))]

In [238]:
test5_normal = data[(data['device']==test5device)&(data.malware=='normal')]['data'].tolist()
test5_normal = scaled.transform(test5_normal).tolist()
y5_normal =  [1 for i in range(0, len(test5_normal))]

In [239]:
for da in [X_train, X_val, X_test, test1_normal, test1_attack, test2_normal, test2_attack, test3_normal, test4_normal, test5_normal]:
    for d in da:
        d.pop(6)
        d.pop(0)

In [240]:
model = OneClassSVM(cache_size=200, gamma='scale', kernel='rbf',nu=0.01,  shrinking=True,  tol=1e-6,verbose=False)

In [241]:
class Autoencoder_DNN(T.nn.Module):  # input_len-32-8-32-input_len
  def __init__(self, input_len):
    super(Autoencoder_DNN, self).__init__()
    self.fc1 = T.nn.Linear(input_len, 64)
    self.fc2 = T.nn.Linear(64, 16)
    self.fc3 = T.nn.Linear(16, 8)
    self.fc4 = T.nn.Linear(8, 16)
    self.fc5 = T.nn.Linear(16, 64)
    self.fc6 = T.nn.Linear(64, input_len)

  def encode(self, x):  # input_len-32-8
    z = T.relu(self.fc1(x))
    z = T.relu(self.fc2(z)) 
    z = T.relu(self.fc3(z)) 
    return z  

  def decode(self, x):  # 8-32-input_len
    z = T.relu(self.fc4(x))
    z = T.relu(self.fc5(z)) 
    z = T.relu(self.fc6(z)) 
    return z
    
  def forward(self, x):  # 65-32-8-32-65
    z = self.encode(x) 
    z = self.decode(z) 
    return z  # in [0.0, 1.0]

In [242]:
def train(train_loader, val_loader, input_len):
    net = Autoencoder_DNN(input_len)
    net = net.train() 
    loss_func = T.nn.MSELoss()
    optimizer = T.optim.Adam(net.parameters(), lr=0.01)
    max_epochs = 50
    # print("Starting training")
    last_val_loss = float("inf")
    patience = 0
    last_loss = 0
    for epoch in range(0, max_epochs):
        loss = 0
        # if epoch > 0 and epoch % (max_epochs/10) == 0:
        #     print("epoch = %6d" % epoch, end="")
        #     print(" prev total loss = %7.4f, perv total val-loss = %7.4f" %( last_loss,val_loss))
        for curr_bat in train_loader:
            X = T.Tensor(curr_bat)
            optimizer.zero_grad()
            oupt = net(X)
            loss_obj = loss_func(oupt, X)  # note X not Y
            loss += loss_obj.item()
            loss_obj.backward()
            optimizer.step()
        last_loss = loss
        val_loss = 0
        with T.no_grad():
            for curr_bat in val_loader:
                X = T.Tensor(curr_bat)
                oupt = net(X)
                val_loss_obj = loss_func(oupt, X)  # note X not Y
                val_loss += val_loss_obj.item()
            # print(loss, val_loss)
            if val_loss < last_val_loss:
                last_val_loss = val_loss
                patience = 0
            else:
                patience += 1
        if patience >= 10:
            break               
    # print("Training stop at epoch： %d" %epoch)
    return net

def find_threshold(net, X_train):
    net = net.eval()
    loss_func = T.nn.MSELoss()
    with T.no_grad():
        x_t = T.Tensor(X_train)
        y_t = net(x_t)
        y_pred = np.array([loss_func(y_t[i],x_t[i]).item() for i in range(0,len(x_t))])
    down_threshold = np.percentile(y_pred, 2.5)
    up_threshold = np.percentile(y_pred, 97.5)
    return down_threshold, up_threshold

def test(net, X, down_threshold, up_threshold):
    net = net.eval()  
    loss_func = T.nn.MSELoss()
    y_pred = []
    with T.no_grad():
        for i in range(0,len(X)):
            x = T.Tensor(X[i])
            y_t = net(x)
            t = loss_func(y_t,x).item()
            if t > down_threshold and t < up_threshold:
                y_pred.append(1)
            else:
                y_pred.append(-1)
    return y_pred

In [243]:
# X_train = T.FloatTensor(X_train)
# X_val = T.FloatTensor(X_val)
# train_loader = T.utils.data.DataLoader(X_train, batch_size=256, shuffle=False, drop_last=False, pin_memory=True, num_workers=2)
# val_loader = T.utils.data.DataLoader(X_val, batch_size=256, shuffle=False, drop_last=False, num_workers=2)
# input_len = len(X_train[0])

In [244]:
# net = train(train_loader,val_loader, input_len)

In [245]:
# down_threshold, up_threshold = find_threshold(net, X_train)
# y_pred = test(net, X_val, down_threshold, up_threshold)
# val_score = metrics.accuracy_score(y_val,y_pred)

In [246]:
# metrics.accuracy_score(test(net, X_val, down_threshold, up_threshold), y_val), metrics.accuracy_score(test(net, X_test, down_threshold, up_threshold), y_test)

In [247]:
# metrics.accuracy_score(test(net, test1_normal, down_threshold, up_threshold), y1_normal), metrics.accuracy_score(test(net, test1_attack, down_threshold, up_threshold), y1_attack)

In [248]:
# metrics.accuracy_score(test(net, test2_normal, down_threshold, up_threshold), y2_normal), metrics.accuracy_score(test(net, test2_attack, down_threshold, up_threshold), y2_attack)

In [249]:
# metrics.accuracy_score(test(net, test3_normal, down_threshold, up_threshold), y3_normal)

In [250]:
# metrics.accuracy_score(test(net, test4_normal, down_threshold, up_threshold), y4_normal)

In [251]:
# metrics.accuracy_score(test(net, test5_normal, down_threshold, up_threshold), y5_normal)

In [252]:
# model =make_pipeline(
#             Nystroem(gamma=0.6, random_state=42, n_components=150),
#             SGDOneClassSVM(
#                 nu=0.05,
#                 shuffle=True,
#                 fit_intercept=True,
#                 random_state=42,
#                 tol=1e-6,
#             ),
#         )

In [253]:
model.fit(X_train)

OneClassSVM(nu=0.01, tol=1e-06)

In [254]:
metrics.accuracy_score(model.predict(X_val), y_val), metrics.accuracy_score(model.predict(X_test), y_test)

(0.8703703703703703, 1.0)

In [255]:
metrics.accuracy_score(model.predict(test1_normal), y1_normal), metrics.accuracy_score(model.predict(test1_attack), y1_attack)

(0.0, 1.0)

In [256]:
metrics.accuracy_score(model.predict(test2_normal), y2_normal), metrics.accuracy_score(model.predict(test2_attack), y2_attack)

(0.8916666666666667, 0.9965277777777778)

In [257]:
metrics.accuracy_score(model.predict(test3_normal), y3_normal)

0.0

In [258]:
metrics.accuracy_score(model.predict(test4_normal), y4_normal)

0.6527777777777778

In [259]:
metrics.accuracy_score(model.predict(test5_normal), y5_normal)

0.8944444444444445